<a href="https://colab.research.google.com/github/ChristopherMajor/DS-Unit-2-Linear-Models/blob/master/module2-regression-2/Christopher_Major_LS_DS_212_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [3]:
df['created'].tail()

49347    2016-06-02 05:41:05
49348    2016-04-04 18:22:34
49349    2016-04-16 02:13:40
49350    2016-04-08 02:13:33
49351    2016-04-12 02:48:07
Name: created, dtype: object

In [0]:
#make created column dt format
df['created']= pd.to_datetime(df['created'], infer_datetime_format=True)

#making new column with just month value
df['month'] = df['created'].dt.month

In [5]:
df['month']

0        6
1        6
2        4
3        4
4        4
        ..
49347    6
49348    4
49349    4
49350    4
49351    4
Name: month, Length: 48817, dtype: int64

In [0]:
#set conditions for training data to include only april and may
condition = ((df['month'] == 4)|
             (df['month'] == 5))
train = df[condition]

In [7]:
#making sure only april and may are in data
train['month'].describe()

count    31844.000000
mean         4.490736
std          0.499922
min          4.000000
25%          4.000000
50%          4.000000
75%          5.000000
max          5.000000
Name: month, dtype: float64

In [8]:
#setting up test set
condition2 = (df['month']==6)
test=df[condition2]
test['month'].describe()

count    16973.0
mean         6.0
std          0.0
min          6.0
25%          6.0
50%          6.0
75%          6.0
max          6.0
Name: month, dtype: float64

In [9]:
train.shape, test.shape

((31844, 35), (16973, 35))

In [0]:
from sklearn.linear_model import LinearRegression

In [11]:
model = LinearRegression()
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,month
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4


In [0]:
import pandas as pd
import plotly.express as px

In [0]:
#arrange y target vectors
target= 'price'
y_train = train[target]
y_test = test[target]

In [0]:
#arrange X features matrices
features = ['latitude',
            'longitude']

X_train = train[features]
X_test = test[features]


In [0]:
# px.scatter_3d(train,
#               x='latitude',
#               y='longitude',
#               z='price',
#               title='Apartment Prices in NY from May to April 2016')

#this for some reason crashed my notebook. i ran the 3dscatter in lecture notebook it worked fine. 
#but after this one crashed its not showing up either idk. 

In [15]:
#fit model and calculate mae for training data
from sklearn.metrics import mean_absolute_error
model.fit(X_train, y_train)
y_pred=model.predict(X_train)
mae=mean_absolute_error(y_train, y_pred)
print(mae)

1147.1493278231892


In [17]:
#trying model on test data
y_pred = model.predict(X_test)
mae=mean_absolute_error(y_test, y_pred)
print(f'Test Error: {mae:.2f} Dollars')

Test Error: 1139.70 dollars


In [19]:
#model coefficient and intercepts.
model.intercept_, model.coef_
#hard to interpret because the latitudes are negative as well. 
#i think higher north it adds to value, then moving west increases values too.

(-1285931.9851655478, array([  2208.18971896, -16215.70541389]))

In [0]:
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score

In [0]:
def squared_errors(df, features,target,m,b):
  #make predictions 
  x=df[features]
  y=df[target]
  y_pred = m*x+b

  #print regression metrics
  mse=mean_squared_error(y, y_pred)
  rmse=np.sqrt(mse)
  mae=mean_absolute_error(y ,y_pred)
  r2=r2_score(y, y_pred)
  print('Mean Squared Error', mse)
  print('Root Mean Squared Error:', rmse)
  print('Mean Absolute Error', mae)
  print('R^2:', r2)


In [30]:
#finding squared errors for training data set
features= ['dogs_allowed']
squared_errors(train, features, target, m=0, b=y_train.mean())

Mean Squared Error 3105028.217891242
Root Mean Squared Error: 1762.1090255404863
Mean Absolute Error 1201.8811133682555
R^2: 0.0


In [31]:
#error for test data
squared_errors(test, features, target, m=0, b=y_test.mean())

Mean Squared Error 3108021.267852562
Root Mean Squared Error: 1762.9581015590138
Mean Absolute Error 1200.8695333832386
R^2: 0.0


In [41]:
#trying other m and b values

squared_errors(test, features, target, m=2000, b=y_test.mean())

Mean Squared Error 4703342.8923155125
Root Mean Squared Error: 2168.7191824474444
Mean Absolute Error 1705.7195843003835
R^2: -0.513291733542484
